In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data/FastCheckTUData.csv', encoding='utf-8')

In [3]:

# Constants for Denmark Bounding Box (Rough approximation)
DK_MIN_LAT, DK_MAX_LAT = 54.5, 57.8
DK_MIN_LON, DK_MAX_LON = 8.0, 15.2

def haversine_np(lat1, lon1, lat2, lon2):
    """
    Vectorized Haversine distance calculation (returns km).
    """
    R = 6371.0  # Earth radius in kilometers

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def check_bounds(df, lat_col, lon_col):
    """Returns a boolean Series: True if inside Denmark, False otherwise."""
    return (
        (df[lat_col] >= DK_MIN_LAT) & (df[lat_col] <= DK_MAX_LAT) &
        (df[lon_col] >= DK_MIN_LON) & (df[lon_col] <= DK_MAX_LON)
    )

def validate_speeds(df):
    """
    Check implied speed vs transport mode limits.
    Returns a boolean series: True if speed is IMPOSSIBLE/HARD WRONG.
    """
    # Calculate implied speed (km/h)
    # Avoid division by zero
    duration_hours = df['stagedurationmin_raw'] / 60.0
    implied_speed = df['stagelength_raw'] / duration_hours.replace(0, np.nan)
    
    # Define max realistic speeds (km/h) with a buffer
    # Mode 1: Walk, 2: Bike, 3-25: Car/Motorized
    
    # Initialize flags
    is_speed_impossible = pd.Series(False, index=df.index)
    
    # 1. Walking: Flag if > 15 km/h (world record pace buffer)
    walk_mask = (df['transportmiddel'] == 1)
    is_speed_impossible |= (walk_mask & (implied_speed > 15))
    
    # 2. Biking: Flag if > 60 km/h (fast e-bike downhill buffer)
    bike_mask = (df['transportmiddel'] == 2)
    is_speed_impossible |= (bike_mask & (implied_speed > 60))
    
    # 3. Car/Motorized: Flag if > 180 km/h
    car_mask = (df['transportmiddel'].between(3, 25))
    is_speed_impossible |= (car_mask & (implied_speed > 180))

    return is_speed_impossible

def perform_fast_validation(df):
    """
    Main driver for the fast pass. 
    Adds flag columns and a 'classification' column.
    """
    print("--- Starting Fast Validation Pass ---")
    
    # 1. Geo Check (Haversine) - Trip Distance
    df['calc_dist_geo'] = haversine_np(
        df['start_lat'], df['start_lon'], 
        df['til_lat'], df['til_lon']
    )

    # 2. Anchor Checks (Home/Work)
    # Handle NaNs automatically (Haversine returns NaN if input is NaN)
    df['dist_start_home'] = haversine_np(df['start_lat'], df['start_lon'], df['home_lat'], df['home_lon'])
    df['dist_end_home']   = haversine_np(df['til_lat'],   df['til_lon'],   df['home_lat'], df['home_lon'])
    
    # 3. Bounding Box Checks
    start_ok = check_bounds(df, 'start_lat', 'start_lon')
    end_ok   = check_bounds(df, 'til_lat', 'til_lon')
    
    # 4. Speed Checks
    flag_speed_impossible = validate_speeds(df)
    
    # --- FLAGGING LOGIC ---
    
    # Flag: Coordinates outside Denmark or (0,0)
    df['flag_coords_bad'] = (~start_ok) | (~end_ok)
    
    # Flag: Impossible speed
    df['flag_speed_bad'] = flag_speed_impossible
    
    # Flag: Zero distance trip but not marked as such? (Optional sanity check)
    df['flag_zero_dist'] = (df['calc_dist_geo'] < 0.005) & (df['stagelength_raw'] > 1.0)

    # --- CLASSIFICATION ---
    # Default to trusted
    df['validation_status'] = 'trusted'
    
    # Identify Hard Wrongs
    # Criteria: Bad coords OR Impossible Speed
    hard_wrong_mask = (df['flag_coords_bad']) | (df['flag_speed_bad'])
    df.loc[hard_wrong_mask, 'validation_status'] = 'hard_wrong'
    
    # (Optional) Mild Suspicion logic could go here
    # e.g., if dist_start_home > 200km implies suspicious context, etc.
    
    return df


In [4]:
# Run the fast validation on the dataframe
df = perform_fast_validation(df)

--- Starting Fast Validation Pass ---


In [5]:
# Save the validated dataframe as a checkpoint
df.to_csv('validated_df_checkpoint.csv', index=False)
print("Validated dataframe saved to 'validated_df_checkpoint.csv'")

Validated dataframe saved to 'validated_df_checkpoint.csv'


In [6]:
print("Validation Status Counts:")
print(df['validation_status'].value_counts())

print("\nExample Hard Wrong Rows:")
df[df['validation_status'] == 'hard_wrong'].head()

Validation Status Counts:
validation_status
trusted       228204
hard_wrong      7560
Name: count, dtype: int64

Example Hard Wrong Rows:


,SessionId,homeText_raw,workText_raw,startDayText_raw,startstedadrsogeord,startTripText_raw,daystartmuncode,turid,tiladrsogeord,tiladrtext_raw,...,start_lon,til_lat,til_lon,calc_dist_geo,dist_start_home,dist_end_home,flag_coords_bad,flag_speed_bad,flag_zero_dist,validation_status
107,347096,"Plantagevej 1, 4941 Bandholm","Kofoedsminde - Udviklingscentret, Højbovej 6, ...","Plantagevej 1, 4941 Bandholm",NaN,"kofoedsminde - udviklingscentret, højbovej 6, ...",NaN,2118225,"Region Sjællands Hovedkontor, Rødbyhavn","kofoedsmindes hovedkontor, rødbyhavn",...,11.351456,0.000000,4.511256,6110.319369,20.454239,6130.283579,True,False,False,hard_wrong
108,347096,"Plantagevej 1, 4941 Bandholm","Kofoedsminde - Udviklingscentret, Højbovej 6, ...","Plantagevej 1, 4941 Bandholm","Region Sjællands Hovedkontor, Rødbyhavn","kofoedsmindes hovedkontor, rødbyhavn",NaN,2118226,NaN,"kofoedsminde - udviklingscentret, højbovej 6, ...",...,4.511256,54.662786,11.351456,6110.319369,6130.283579,20.454239,True,False,False,hard_wrong
150,347115,"Toftevangen 77, 4130 Viby Sjælland",NaN,NaN,NaN,"toftevangen 77, 4130 viby sjælland",NaN,2118279,Gelsted 4160,stenagergårdsvej\r\ngelsted\r\n4130 herlufmagle,...,12.032662,0.000000,4.511256,6213.935173,0.000000,6213.935173,True,False,False,hard_wrong
151,347115,"Toftevangen 77, 4130 Viby Sjælland",NaN,NaN,Gelsted 4160,stenagergårdsvej\r\ngelsted\r\n4130 herlufmagle,NaN,2118280,NaN,"toftevangen 77, 4130 viby sjælland",...,4.511256,55.545711,12.032662,6213.935173,6213.935173,0.000000,True,False,False,hard_wrong
227,347147,"Kejlstrupvej 15, 8600 Silkeborg",NaN,NaN,NaN,"kejlstrupvej 15, 8600 silkeborg",NaN,2118373,"Nørrevænget 42, 8600","nørrevænget 42, 8600 silkeborg",...,9.554954,56.183815,9.545761,0.789938,0.000000,0.789938,False,True,False,hard_wrong


In [7]:
import correctors

In [8]:
import importlib

In [9]:
hard_wrong_df = df[df['validation_status'] == 'hard_wrong']

print(f"Processing {len(hard_wrong_df)} hard_wrong rows")

corrections = correctors.repair_hard_wrongs(hard_wrong_df)

print("Validation status in corrections:")
print(corrections['validation_status'].value_counts())

print("Corrections completed.")

Processing 7560 hard_wrong rows
--- Starting Correction Pipeline on 7560 rows ---


Correcting rows:   0%|          | 0/7560 [00:00<?, ?it/s]

Loading Address Index...
Building Optimized Postcode Index...
Index Built. Retriever Ready.


Correcting rows:   1%|          | 88/7560 [01:58<4:16:45,  2.06s/it] 

Saved checkpoint at row 3700


...

Saved checkpoint at row 230000


Correcting rows:  98%|█████████▊| 7440/7560 [2:05:50<00:46,  2.56it/s]

Saved checkpoint at row 232500


Correcting rows: 100%|██████████| 7560/7560 [2:07:36<00:00,  1.01s/it]


--- Finished. Corrected 7341 rows. Coords fixed: 6931, OSRM attempted: 7529 ---
Validation status in corrections:
validation_status
corrected     7341
hard_wrong     219
Name: count, dtype: int64
Corrections completed.


In [10]:
df['corrected_dist_km'] = np.nan
df['corrected_time_min'] = np.nan
df['note_correction'] = np.nan

df.update(corrections)

print("DataFrame updated with corrections.")

DataFrame updated with corrections.


C:\Users\nicoc\AppData\Local\Temp\ipykernel_10272\3699952109.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.update(corrections)


In [11]:
df.columns

Index(['SessionId', 'homeText_raw', 'workText_raw', 'startDayText_raw',
       'startstedadrsogeord', 'startTripText_raw', 'daystartmuncode', 'turid',
       'tiladrsogeord', 'tiladrtext_raw', 'transportmiddel', 'stagelength_raw',
       'stagedurationmin_raw', 'home_lat', 'home_lon', 'work_lat', 'work_lon',
       'start_lat', 'start_lon', 'til_lat', 'til_lon', 'calc_dist_geo',
       'dist_start_home', 'dist_end_home', 'flag_coords_bad', 'flag_speed_bad',
       'flag_zero_dist', 'validation_status', 'corrected_dist_km',
       'corrected_time_min', 'note_correction'],
      dtype='object')

In [12]:
# Save the final updated dataframe
df.to_csv('DET_final_corrected_df.csv', index=False)
print("Final corrected dataframe saved to 'DET_final_corrected_df.csv'")

Final corrected dataframe saved to 'DET_final_corrected_df.csv'


In [13]:
print('Total rows in df:', len(df))
print('NaN in validation_status:', df['validation_status'].isna().sum())
print('Sum of counts:', df['validation_status'].value_counts().sum())

Total rows in df: 235764
NaN in validation_status: 0
Sum of counts: 235764


In [ ]:
# Compare validated_df_checkpoint and final_corrected_df
#validated_df = pd.read_csv('validated_df_checkpoint.csv')
final_df = pd.read_csv('DET_final_corrected_df.csv')

#print("Unique SessionId in validated_df_checkpoint:", validated_df['SessionId'].nunique())
print("Unique SessionId in final_corrected_df:", final_df['SessionId'].nunique())

print("\nValidation status in validated_df_checkpoint:")
#print(validated_df['validation_status'].value_counts())
print("\nValidation status in final_corrected_df:")
print(final_df['validation_status'].value_counts())

# Check if there are differences in the data
#differences = (validated_df != final_df).any(axis=1).sum()
#print(f"\nNumber of rows with differences: {differences}")

# Specifically for hard_wrong rows
#hard_wrong_original = validated_df[validated_df['validation_status'] == 'hard_wrong']
hard_wrong_final = final_df[final_df['validation_status'] == 'hard_wrong']
#print(f"Hard wrong rows in original: {len(hard_wrong_original)}")
print(f"Hard wrong rows in final: {len(hard_wrong_final)}")

Unique SessionId in final_corrected_df: 74596

Validation status in validated_df_checkpoint:

Validation status in final_corrected_df:
validation_status
trusted       228204
corrected       7371
hard_wrong       189
Name: count, dtype: int64
Hard wrong rows in final: 189
